# Projeto prático de software abero e programação em SIG
# METEO2MAPS <img src="iconsmeteo2map.png" width=30 height=30/>
### João H. Oliveira, 2021

In [ ]:
import os, urllib.request, json, subprocess, geopandas, pandas, shutil, psycopg2, re, requests
from datetime import datetime
from psycopg2 import extras as psy2extras
from geo.Geoserver import Geoserver

In [ ]:
os.environ['SHAPE_ENCODING'] = "utf-8"
basePath = 'C:\\saprog\\projeto'
outputPath = 'C:\\saprog\\projeto\\output'
os.chdir(basePath)

def deleteAndCreateOutput():
    print ('A limpar e criar directoria de outputs...')
    outDir = os.path.join(basePath, 'output')
    if os.path.isdir(outDir):
        shutil.rmtree(outDir)
    os.mkdir(outDir)

deleteAndCreateOutput()

In [ ]:
print('A ler shapefile...')
caop = geopandas.read_file('caop.shp',encoding='utf-8')
# caop.plot()
# caop.head()
# gdf = caop.loc[caop['Concelho'] == 'Bombarral']
# gdf.head()

In [ ]:
print('A executar Dissolve...')
distritos = caop.dissolve(by = 'Distrito')
# distritos.plot()
# distritos.head(1)

In [ ]:
print('A extrair centroides das regiões admninistrativas...')
shpOutput = os.path.join(outputPath, 'centroides.shp')

# dataProj = distritos.to_crs(epsg = 3763)
# centroides = dataProj.centroid
centroides = distritos.centroid
centroides.to_file(shpOutput, encoding='utf-8')
# centroides.plot()
# centroides.head()

coordx = centroides.x.to_dict()
coordy = centroides.y.to_dict()
# print(coordx)
# print(coordy)

In [ ]:
def getCoordinates(dicX, dicY):
    '''
    Extração de coordenadas geográficas úteis ao harvest de dados meteorológicos.
    Devolve um dicionário onde as chaves são os distritos, à qual está associado um tuplo com as coordenadas Lat Long.
    '''

    coord = [dicX, dicY]
    coordDic = {}
    for i in dicX.keys():
        coordDic[i] = tuple(coordDic[i] for coordDic in coord)
    return coordDic

print('A extrair coodenadas geográficas...')
coord = getCoordinates(coordx, coordy)
# print(coord)

In [ ]:
# Carregamento de shapefile "centroides" na bd
command = ["C:\\OSGeo4W64\\bin\\ogr2ogr.exe",
          "-f", "PostgreSQL",
          "PG:host=localhost user=postgres dbname=meteo password=3763", outputPath,
          "-lco", "GEOMETRY_NAME=the_geom", "-lco", "FID=gid", "-lco",
          " PRECISION=no", "-nlt", "PROMOTE_TO_MULTI", "-nln", "centroides", "-overwrite"]
subprocess.check_call(command)

In [ ]:
apiKey = '44cfad7f82ec61d3f420de3201b703d4'

def getWeather(coordDic, apikey):
    '''
    Função para recolha dos dados meteorológicos provenientes da OpenWeather One Call API.
    Devolve uma dataframe das váriáveis meteorológicas por distrito.
    '''
    forecast = []
    for item in coord.items():
        lat = str(item[1][1])
        long = str(item[1][0])
        url = 'https://api.openweathermap.org/data/2.5/onecall?lat=' + lat + '&lon=' + long + \
            '&exclude=minutely,hourly,daily,alerts&appid=' + apiKey + '&units=metric'
        with urllib.request.urlopen(url) as url:
            data = json.loads(url.read().decode())
            districtForecast = {}
            districtForecast['distrito'] = item[0]
            districtForecast['forecast_date'] = datetime.utcfromtimestamp(data.get('current').get('dt')).strftime('%d-%m-%Y')
            districtForecast['forecast_time'] = datetime.utcfromtimestamp(data.get('current').get('dt')).strftime('%H:%M:%S')
            main = data.get('current').get('weather')
            for item in main:
                districtForecast['weather_desc'] = item.get('main')
            districtForecast['temperature'] = data.get('current').get('temp')
            districtForecast['feels_like'] = data.get('current').get('feels_like')
            districtForecast['pressure'] = data.get('current').get('pressure')
            districtForecast['humidity'] = data.get('current').get('humidity')
            districtForecast['dew_point'] = data.get('current').get('dew_point')
            districtForecast['ultrav_index'] = data.get('current').get('uvi')
            districtForecast['wind_speed'] = data.get('current').get('wind_speed')
            districtForecast['wind_deg'] = data.get('current').get('wind_deg')
            forecast.append(districtForecast)
    forecast_df = pandas.DataFrame(forecast)
    return forecast_df

forecast_df = getWeather(coord, apiKey)
# forecast_df.head(-1)

In [ ]:
password = open(os.path.join('pw.txt'), 'r').readline()
password = str(password)

con = psycopg2.connect(dbname='meteo', user='postgres', password=password, host='localhost', port='5432')

def df2PgSQL(conn, df, table):
    '''
    Utilização da função psycopg2.extras.execute_values()
    para carregamento da data frame colhida na tabela PostGreSQL "forecast"
    '''
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        psy2extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("dfToPgSQL() done")
    cursor.close()

df2PgSQL(con, forecast_df, 'forecast')

In [ ]:
# # Carregamento de shapefile "caop" na bd
# command = ["C:\\OSGeo4W64\\bin\\ogr2ogr.exe",
#           "-f", "PostgreSQL",
#           "PG:host=localhost user=postgres dbname=meteo password=3763",
#           outputPath,
#           "-lco", "GEOMETRY_NAME=the_geom", "-lco", "FID=gid", "-lco",
#           " PRECISION=no", "-nlt", "PROMOTE_TO_MULTI", "-nln", "caop", "-overwrite"]

# subprocess.check_call(command)

pgConnectionPar = {'host':'localhost', 'user':'postgres', 'dbname':'meteo', 'password':'3763'}

def shp2PgSQL(shp, pgconn):
    command = ["C:\\OSGeo4W64\\bin\\ogr2ogr.exe",
            "-f", "PostgreSQL",
            "PG:host={} user={} dbname={} password={}".format(pgconn.get('host'),\
            pgconn.get('user'), pgconn.get('dbname'), pgconn.get('password')),
            outputPath,
            "-lco", "GEOMETRY_NAME=the_geom", "-lco", "FID=gid", "-lco",
            " PRECISION=no", "-nlt", "PROMOTE_TO_MULTI", "-nln", shp, "-overwrite"]
    exe = subprocess.Popen(command,stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
    for line in exe.stdout:
        line = str(line.decode('utf-8'))
        line_txt = re.sub("\r?\n", '', line)
        print(line_txt)

shpToPgSQL('caop', pgConnectionPar)


In [ ]:
# falta criar código para produção de view dentro da base de dados + join com caop poligonal

In [ ]:
# Initialize the library
geo = Geoserver('http://localhost:8080/geoserver', username='admin', password='geojoao')

# For creating workspace
geo.create_workspace(workspace='sapsig_meteo')